# Big 5 European Leagues - Fixtures Data

**Data Source:** https://www.football-data.co.uk/data.php

In this notebook, I have scraped data from the source website using the Beautiful Soup package. I intend to utilize this code to pull data directly from the source website, save it to a CSV file, and perform analysis on the data.

I am extracting the entire data from all the seasons, beginning from 1993-94 until 2022-23.

**Big 5 European Leagues:**
- Premier League (England)
- Bundesliga (Germany)
- Serie A (Italy)
- Ligue 1 (France)
- La Liga (Spain)

# 1. Loading the libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import os
import pandas as pd
import time
from datetime import date
from pandas.errors import ParserError

# 2. Selecting countries

In [ ]:
# Selecting countries of the leagues for which I want to extract the data.

countries = ["england", "germany", "italy", "france", "spain"]

# 3. Assigning the URL

In [ ]:
# Assigning parts of URL to various variables to be used later in the Beautiful Soup functions.

base_url = "https://www.football-data.co.uk/" 
end_url = "m.php"
start_of_url = "mmz4281/"

# 4. Selecting Seasons

In [ ]:
# Creating a list of seasons for which the data will be extracted only.

start_date = date(2023, 7, 1)
start_year = start_date.year
current_date = date.today()
current_year = current_date.year
number_of_years = current_year - start_year
seasons = []

for i in range(number_of_years + 1):
    season_concat = str(start_year + i)[-2:] + str(start_year + i + 1)[-2:]
    seasons.append(season_concat)

# 5. Using Beautiful Soup

In [ ]:
# Using Beautiful Soup to identify the list of links from where the data will be extracted.

all_links_joined = []
for country in countries:
# Requesting the URL
    country_url = urllib.parse.urljoin(base_url, (country + end_url))
    data = requests.get(country_url)
# Creating object of BeautifulSoup
    soup = BeautifulSoup(data.text)
# Identifying all the links from where data will be extracted.
    all_links = [link["href"] for link in soup.find_all("a") 
                          if link["href"].startswith(start_of_url)
                          and link["href"][-4:] == ".csv" 
                          and link["href"][-6:] != "E2.csv" 
                          and link["href"][-6:] != "E3.csv" 
                          and link["href"][-6:] != "EC.csv"]
# Identifying all the links according to the seasons criteria.
    for link in all_links:
        joined_url = urllib.parse.urljoin(base_url, link)
        year = link.split("/")[-2]
        if not year in seasons:
            all_links_joined.append(joined_url)
        else:
            continue

In [ ]:
# List of all the links from where data will be extracted.

all_links_joined

In [ ]:
# The total number of links from where data will be extracted.

len(all_links_joined)

# 6. Initializing the format of the dataframe.

In [ ]:
# Uploading the data from the first link to a pandas dataframe. To be referenced later to append other fixtures data properly.

# Changing the format of the season value
year_for_df = all_links_joined[0].split("/")[-2]
year_for_df = year_for_df[:2] + "-" + year_for_df[-2:]

# Uploading the data to the dataframe using pandas and inserting a column named "Season" to the dataframe.
df_concat = pd.read_csv(all_links_joined[0])
df_concat.insert(loc = 0, column = "Season", value = year_for_df)

# 7. Appending the entire data.

In [ ]:
# Appending the data together from all the links.

for link in all_links_joined[1:]:
    filename = link.split("/")[-1]
    year = link.split("/")[-2]
    year_filename = year + filename
    try:
# Reading the CSV file to append the data.
        df = pd.read_csv(link, encoding = "unicode_escape")
# To delete the duplicate columns in some of the files. 
        df = df.loc[:, ~df.columns.duplicated()].reset_index(drop = True)
# To resolve the tokenization error in some of the files due to mismatch of number of columns in some of the rows.
    except ParserError:
        df = pd.read_csv(link, encoding = "unicode_escape", sep = "\t")
# Saving the CSV file to correct the errors.
        df.to_csv(year_filename, index = False)
# Opening the CSV file and calculating the maximum number of columns.
        with open(year_filename, "r") as temp_f:
            col_count = [len(l.split(",")) for l in temp_f.readlines()]
        column_names = [i for i in range(max(col_count))]        
# Reading the CSV file again to append the correct data.
        df = pd.read_csv(link, encoding = "unicode_escape", names = column_names)
# Correcting the headers of the file
        headers = df.iloc[0].values
        df.columns = headers
        df.drop(index = 0, axis = 0, inplace = True)
        os.remove(year_filename)
# To delete the duplicate columns in some of the files.
        df = df.loc[:, ~df.columns.duplicated()].reset_index(drop = True)
# To keep a track of the progress of the data extraction from the links.
    print(year)
# Inserting a column named "Season" to the dataframe.
    df.insert(loc = 0, column = "Season", value = (year[:2] + "-" + year[-2:]))
# Appending the data to the dataframe.
    df_concat = pd.concat([df_concat, pd.DataFrame(df)], ignore_index = True)  
    time.sleep(1)
# Removing all the null rows
df_concat.dropna(subset = "Div", axis = 0, inplace = True)
# Removing all the null columns and irrelevant columns
df_concat = df_concat.iloc[:, :25]
# Renaming the league names
df_concat["Div"] = df_concat["Div"].replace({"E0": "Premier League", "E1": "EFL Championship",
                                             "D1": "Bundesliga", "D2": "Bundesliga 2",
                                             "I1": "Serie A", "I2": "Serie B",
                                             "SP1": "LaLiga", "SP2": "LaLiga 2",
                                             "F1": "Ligue 1", "F2": "Ligue 2"})

# Saving the extracted data from all the links to CSV file
df_concat.to_csv("past-data.csv", index = False)

In [ ]:
# Checking the number of matches per season to ensure the data extracted is correct.

season_matches_count = df_concat.groupby(["Season", "Div"]).count().iloc[:, 0].reset_index()
season_matches_count

# End